# 머신러닝 with Python 
## 09 앙상블 학습 보팅 연습2- wine data set

##### 노트를 활용해 직접 다른 데이터로 해보았다.

책 p.259~ (원본 데이터 - 붓꽃 분류, 이번 사용 데이터 와인 데이터), 일부는 랜덤포레스트 실습 부분 참조

### 데이터 불러오기

In [1]:
from sklearn import datasets 

In [5]:
raw_wine = datasets.load_wine() # 와인 데이터 가져오기

In [6]:
# 데이터 살펴보기

raw_wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

### 피처, 타깃 데이터 지정

보통 train data는 행렬이므로 대문자 X, 예측되어야 할 값은 스칼라로 소문자 y가 된다는 것을 들은게 생각이 나부렀다.}

In [7]:
X = raw_wine.data
y = raw_wine.target

### 트레이닝/테스트 데이터 분할

In [8]:
from sklearn.model_selection import train_test_split              # 분할을 위해 필요한 함수
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0) # 분리. randomstate는 고정

### 데이터 표준화

In [9]:
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위한 함수
std_scale = StandardScaler()                     # 표준화 스케일러 지정
std_scale.fit(X_tn)                              # 트레이닝 피처를 기준으로 표준화를 적합시키기


X_tn_std = std_scale.transform(X_tn) 
X_te_std = std_scale.transform(X_te)             # 트레인, 테스트 데이터 각각 적합시킨 표준화에 맞게 변형

### 데이터 학습 - 보팅 학습

이번 실습에서는 로지스틱 회귀분석, 가우시안 나이브 베이즈, 서포트 벡터 머신을 사용한다.

In [10]:
from sklearn.linear_model import LogisticRegression # 로지트틱 회귀분석
from sklearn import svm                             # support vector machine
from sklearn.naive_bayes import GaussianNB          # Gussian Naive Bayes
from sklearn.ensemble import VotingClassifier       # 분류 문제이기 때문이 VotingClassifier를 사용. 회귀문제라면 VotingRegressor를 사용한다.

clf1 = LogisticRegression(multi_class = 'multinomial', 
                          random_state = 1)         # 첫 번째 모형을 로지스틱 회귀분석이라고 정함
clf2 = svm.SVC(kernel = 'linear', 
              random_state = 1)                     # 두 번째 모형을 서포트 벡터 머신 모형이라고 정함
clf3 = GaussianNB()                                 # 세 번째 모형은 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators = [        # estimators : 미리 만든 세 가지 모형을 의미
                                    ('lr', clf1),
                                    ('svm', clf2),
                                    ('gnb', clf3),
                                                ],
                             voting = 'hard',       # voting : hard(default)=과반수가 넘는 라벨, soft=확률이 가장 높은 라벨
                             weights = [1, 1, 1])   # weights : 각 모형의 비율. 중요도 가중치를 어떻게 줄지인 것으로 예상됨. 

clf_voting.fit(X_tn_std, y_tn)                      # 만들어진 모형에 표준화된 트레이닝 피처 데이터와 트레이닝 타깃 데이터를 넣고 적합시킨다.

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('svm', SVC(kernel='linear', random_state=1)),
                             ('gnb', GaussianNB())],
                 weights=[1, 1, 1])

### 데이터 예측

In [11]:
pred_voting = clf_voting.predict(X_te_std)          # std된 테스트 피처 데이터를 넣고 실행
print(pred_voting)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [12]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_voting)        # 실제값과 예측값을 넣음
print(accuracy)

1.0


...?

### confusion matrix 확인

In [13]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_voting)
print(conf_matrix)

[[16  0  0]
 [ 0 21  0]
 [ 0  0  8]]


### 분류 리포트 확인

In [14]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_voting)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



말이 되냐구요... 이러면 안되는 거잖아...